In [1]:
import requests
import pandas
import pprint
from IPython.display import clear_output

In [2]:
def fetch_data(page=None):
    query_params = {"cnt": 'brazil',
                    "q": "A",
                    "type": "song",
#                     "also": ""
    #                 "length|": "0:47"
    #                 "gen": genus,
    #                 "rec": recordist,
    #                 "loc": f'"{location}"' if location else None,  # Location must be wrapped in double quotes.
    #                 "rmk": remarks,
    #                 "lat": latitude,
    #                 "lon": longitude,
    #                 "box": box,
    #                 "nr": catalogue_number,
    #                 "lic": license,
    #                 "area": area,
    #                 "since": since,
    #                 "year": year,
    #                 "month": month
                   }
    query = " ".join([f"{name}:{var}" for name, var in query_params.items() if var])
    r = requests.get('https://www.xeno-canto.org/api/2/recordings', params={"query": query, "page": page})
    return r.json()

In [3]:
def append_to_frame(data_frame, recordings):
    for recording in recordings['recordings']:
        mins_secs = recording['length'].split(':')
        data_frame = data_frame.append({
            'file_id': recording['id'],
            'genus': recording['gen'],
            'species': recording['sp'],
            'name': recording['en'],
            'download_url': 'https://www.xeno-canto.org/{}/download'.format(recording['id']),
            'file_name': recording['file-name'],
            'duration': int(mins_secs[0]) * 60 + int(mins_secs[1]),
            'other_species_in_recording': ', '.join(recording['also'])
        }, ignore_index=True)
    return data_frame

In [4]:
birds = pandas.DataFrame(columns=['file_id', 'genus', 'species', 'name', 'download_url', 'file_name', 'duration', 'other_species_in_recording'])

In [5]:
initial_data = fetch_data()
num_pages = initial_data['numPages']
num_recordings = initial_data['numRecordings']
birds = append_to_frame(birds, initial_data)
for i in range(2, num_pages+1, 1):
    clear_output(wait=True)
    birds = append_to_frame(birds, fetch_data(page=i))
    print('Fetched {} of {} pages.'.format(i, num_pages))

Fetched 23 of 23 pages.


In [6]:
birds = birds[birds['duration'] >= 20][birds['name'] != 'Identity unknown']
birds = birds[birds.groupby('genus').genus.transform(len) > 15].reset_index()

C:\Applications\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [8]:
birds.shape

(6210, 9)

In [9]:
birds.to_csv('birds_dataset.csv')